In [1]:
b1 = bytearray.fromhex("1c0111001f010100061a024b53535009181c")
b2 = bytearray.fromhex("686974207468652062756c6c277320657965")
def xor_hex(x,y):
    result = []
    for i in range(len(x)):
        result.append(x[i] ^ y[i])
    return result    

check = bytes(xor_hex(b1,b2))
print(check.lower())

b"the kid don't play"


In [2]:
def xor_decypher(x):
    import numpy as np
    tsets = bytearray.fromhex(x)
    win_array = np.zeros(256)
    results = [None] * 256
    for i in range(256):
        key_test = [i] * len(tsets)
        result = bytes(xor_hex(tsets,key_test)).lower()
        try:
            results[i] = result.decode("ASCII")
        except UnicodeDecodeError:
            pass
        win_array[i] = scoring(result)
        win_array[i] = 0 if np.isnan(win_array[i]) else win_array[i]

    max_prob = win_array.argmax()
    key = chr(max_prob)
    plaintext = results[max_prob]
    return key,plaintext


In [3]:
occurance_english = {
    'a': 8.2389258,    'b': 1.5051398,    'c': 2.8065007,    'd': 4.2904556,
    'e': 12.813865,    'f': 2.2476217,    'g': 2.0327458,    'h': 6.1476691,
    'i': 6.1476691,    'j': 0.1543474,    'k': 0.7787989,    'l': 4.0604477,
    'm': 2.4271893,    'n': 6.8084376,    'o': 7.5731132,    'p': 1.9459884,
    'q': 0.0958366,    'r': 6.0397268,    's': 6.3827211,    't': 9.1357551,
    'u': 2.7822893,    'v': 0.9866131,    'w': 2.3807842,    'x': 0.1513210,
    'y': 1.9913847,    'z': 0.0746517,    ' ': 100
}

In [4]:
def scoring(x):
    prob = 0
    try:
        for i in x.decode("ASCII"):
            if i in occurance_english.keys():
                prob += occurance_english[i]
        return prob
    except UnicodeDecodeError:
        pass

In [6]:
def edit_dist(x,y):
    test_str1 = str(x)
    test_str2 = str(y)
    test1_len = len(test_str1)
    test2_len = len(test_str2)
    loop_len = test1_len if test1_len < test2_len else test2_len
    diff = 0
    for i in range(loop_len):
        temp_len = len(bin(ord(test_str1[i]))) if len(bin(ord(test_str1[i]))) > len(bin(ord(test_str2[i]))) else len(bin(ord(test_str2[i])))
        if len(bin(ord(test_str1[i]))) != len(bin(ord(test_str2[i]))):
            if len(bin(ord(test_str1[i]))) != 9:
                temp_str1 = bin(ord(test_str1[i]))[:2] + "0" + bin(ord(test_str1[i]))[2:]
                for k in range(temp_len):
                    if temp_str1[k] != bin(ord(test_str2[i]))[k]:
                        diff +=1
            if len(bin(ord(test_str2[i]))) != 9:
                temp_str2 = bin(ord(test_str2[i]))[:2] + "0" + bin(ord(test_str2[i]))[2:]
                for k in range(temp_len):
                    if temp_str2[k] != bin(ord(test_str1[i]))[k]:
                        diff +=1
        else:
            for k in range((temp_len)):
                if bin(ord(test_str1[i]))[k] != bin(ord(test_str2[i]))[k]:
                    diff += 1
    return diff   

In [7]:
import base64
data6 = requests.get('https://cryptopals.com/static/challenge-data/6.txt')

In [8]:
normalized_distances = []
for keysize in range(2,40):
    L1 = (base64.b64decode(data6.text))[:keysize]
    L2 = (base64.b64decode(data6.text))[keysize:keysize * 2]
    L3 = (base64.b64decode(data6.text))[keysize*2:keysize * 3]
    L4 = (base64.b64decode(data6.text))[keysize*3:keysize * 4]
    normalized_dist = float(((edit_dist(L1,L2)) + (edit_dist(L2,L3)) + (edit_dist(L3,L4))) / keysize*3)
    normalized_distances.append((keysize,normalized_dist))
    

normalized_distances = sorted(normalized_distances, key=lambda x:x[1])
print(normalized_distances)

[(2, 27.0), (3, 45.0), (16, 47.4375), (15, 54.4), (29, 56.48275862068965), (28, 56.57142857142857), (30, 57.300000000000004), (33, 57.36363636363636), (32, 57.375), (17, 57.529411764705884), (9, 58.0), (10, 59.099999999999994), (4, 59.25), (24, 59.75), (14, 59.78571428571428), (19, 59.84210526315789), (31, 60.29032258064517), (34, 60.35294117647059), (12, 60.5), (37, 61.216216216216225), (25, 61.32000000000001), (26, 61.38461538461538), (23, 61.69565217391305), (27, 61.77777777777777), (18, 61.83333333333333), (8, 61.875), (36, 62.0), (22, 62.59090909090909), (38, 64.89473684210526), (21, 65.42857142857143), (20, 66.0), (5, 66.6), (35, 66.77142857142857), (7, 68.57142857142857), (6, 71.0), (39, 71.3076923076923), (11, 73.63636363636364), (13, 81.0)]


In [9]:
arr1 =[]
for y in range(0,len((base64.b64decode(data6.text)).hex()),2):
    arr1.append((base64.b64decode(data6.text)).hex()[y:y+2])
    
 

In [11]:
blks = [[] for _ in range(29)]
for i,hexi in enumerate(arr1):
    blks[i%29].append(hexi)
keys = ""
for i in range(29):
    tempstr1 = "".join(ele for ele in blks[i])
    results =xor_decypher(tempstr1)
    keys += results[0]

print(keys)

Terminator X: Bring the noise
